# Train 데이터에 있는 Class를 Label로 학습하여, Test 데이터에 Label을 분류하는 모델 입니다  
이 모델은 GPT를 통해서 학습 할 때 문제 유형을 붙이기에 유용합니다.  
Train 데이터에는 Class 데이터가 있지만, Test 데이터에는 Class가 없기 때문에 이를 분류하기 위한 모델로 국내 공개된 모델 중 하나인 Roberta-Large를 사용해서 학습을 진행하고 Test에 Class labeling을 하는 코드가 있습니다

In [1]:
# 튜닙의 데이터셋이 있는 깃허브를 가져오고, 기본적인 라이브러리를 설치합니다
!git clone https://github.com/tunib-ai/KMWP.git
!pip install transformers sentencepiece spacy datasets

Cloning into 'KMWP'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 35 (delta 3), reused 20 (delta 1), pack-reused 0
Unpacking objects: 100% (35/35), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.2 MB/s 
     |████████████████████████████████| 1.2 MB 87.6 MB/s 
     |████████████████████████████████| 346 kB 85.8 MB/s 
     |████████████████████████████████| 1.1 MB 71.1 MB/s 
     |████████████████████████████████| 596 kB 76.2 MB/s 
     |████████████████████████████████| 86 kB 6.5 MB/s 
     |████████████████████████████████| 6.6 MB 79.1 MB/s 
     |████████████████████████████████| 86 kB 6.9 MB/s 
     |████████████████████████████████| 212 kB 91.5 MB/s 
     |████████████████████████████████| 1.1 MB 76.4 MB/s 
     |████████████████████████████████| 140 kB 83.0 MB/s 
     |███

In [2]:
# Huggingface의 허브를 사용하기 위해서는 Huggingface에 가입하고 본인의 토큰을 사용하셔야 합니다
from huggingface_hub import notebook_login
notebook_login()
!git config --global credential.helper store

Login successful
Your token has been saved to /root/.huggingface/token


# 필요 라이브러리, 데이터 로드

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

import tensorflow as tf

from transformers import AutoConfig, AutoTokenizer, TFAutoModelForSequenceClassification
from transformers.keras_callbacks import PushToHubCallback
from datasets import Dataset

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

tqdm.pandas()
train_path = '/content/KMWP/data/train.csv'
test_path = '/content/KMWP/data/test.csv'
sample_path = '/content/KMWP/data/sample_answersheet.json'
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
sample = pd.read_json(sample_path)

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


이 코드는 코랩의 TPU 사용을 기준으로 작성 되었습니다.  
아래는 TPU를 초기화하고 연결하는 코드입니다.

In [2]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.TPUStrategy(tpu)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.68.136.122:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.68.136.122:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


사용할 모델을 선정하고 토크나이징해서 길이의 분포를 확인합니다.  
여기에선 klue/roberta-large 모델을 사용합니다.

# 데이터 전처리

In [3]:
MODEL_PATH = 'klue/roberta-large'
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

In [4]:
train['p_tokenized'] = train.problem.apply(lambda x: tokenizer.encode(x))
train['p_len'] = train.p_tokenized.apply(len)

In [5]:
train.p_len.describe()

count    2820.000000
mean       38.428014
std        12.260828
min         9.000000
25%        30.000000
50%        36.000000
75%        45.250000
max       118.000000
Name: p_len, dtype: float64

In [6]:
MAX_LEN = 64

In [7]:
# Huggingface의 Dataset 구조를 사용해서 tokenizing 하면 편리하고 빠르게 할 수 있습니다
train_ds = Dataset.from_pandas(train[['problem','class']])

In [8]:
def tokenize_fn(ds):
    tokenized = tokenizer(ds['problem'],max_length=MAX_LEN, padding='max_length', truncation=True, return_tensors='np')
    return dict(
        input_ids=tokenized['input_ids'],
        attention_mask=tokenized['attention_mask'],
        labels=ds['class']
    )

In [9]:
# 정의한 tokenize function으로 각 데이터를 tokenizing
hf_train_ds = train_ds.map(tokenize_fn, batched=True)
# 사용하지 않는 problem, class를 제거
hf_train_ds = hf_train_ds.remove_columns(['problem','class'])

  0%|          | 0/3 [00:00<?, ?ba/s]

In [10]:
hf_train_ds

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 2820
})

roberta-large는 Torch 모델이고, Tensorflow 모델로 불러오기 위해선 from_pt=True  
class 종류는 8개 이므로 num_labels=8  
label의 갯수가 많으므로 problem type을 multi_label_classification으로 정의해야 정상 작동 합니다

# 모델 빌드

In [11]:
# 
# label
def build_model():
    model = TFAutoModelForSequenceClassification.from_pretrained(MODEL_PATH,from_pt=True, num_labels=8, problem_type="multi_label_classification")
    model.compile(
    optimizer=tf.keras.optimizers.SGD(momentum=0.9),
)
    return model

In [12]:
# strategy.scope context 안에서 모델을 정의하고 load해야 정상 작동합니다
with strategy.scope():
    model = build_model()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [13]:
# early stop callback 정의
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
)

In [14]:
# dataset 내부의 데이터 타입을 numpy dtype으로 변경
hf_train_ds.set_format('np')

In [15]:
# scikit-learn의 label encoder를 활용해서 label을 0~7로 바꿔줍니다
label_encoder = LabelEncoder()
encoded_label = label_encoder.fit_transform(hf_train_ds['labels'])

In [16]:
# 클래스의 분포에 맞게 비슷한 비율로 train, valid 인덱스를 추출합니다
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=5959)
train_idx,valid_idx = next(iter(kfold.split(train.problem.to_numpy(),encoded_label)))

In [17]:
# 간혹 모델의 데이터 타입이 맞지 않아 에러가 나는 경우가 있습니다 int32로 바꿔줍니다
train_set = dict(
    input_ids=tf.cast(hf_train_ds['input_ids'][train_idx],dtype=tf.int32),
    attention_mask=tf.cast(hf_train_ds['attention_mask'][train_idx],dtype=tf.int32),
    labels=tf.cast(encoded_label[train_idx],dtype=tf.int32),
)
test_set = dict(
    input_ids=tf.cast(hf_train_ds['input_ids'][valid_idx],dtype=tf.int32),
    attention_mask=tf.cast(hf_train_ds['attention_mask'][valid_idx],dtype=tf.int32),
    labels=tf.cast(encoded_label[valid_idx],dtype=tf.int32),
)

# Data Input Pipeline  
데이터를 모델에 빠르게 공급하기 위한 일련의 과정으로, 자세한 내용은 Tensorflow의 data API를 참고해주세요

In [18]:
AUTO = tf.data.AUTOTUNE
BATCH_SIZE= 32 * 8

def prefetch_ds(ds):
    return ds.repeat().shuffle(2500).batch(BATCH_SIZE,num_parallel_calls=AUTO).prefetch(AUTO)

In [19]:
tf_train_ds = tf.data.Dataset.from_tensor_slices(train_set)
tf_train_ds = prefetch_ds(tf_train_ds)
tf_val_ds = tf.data.Dataset.from_tensor_slices(test_set)
tf_val_ds = prefetch_ds(tf_val_ds)

In [20]:
# 모델 허브에 모델을 업로드 하는 callback으로 학습 과정까지 함께 기록해서 올려줍니다
hub_callback = PushToHubCallback(
    output_dir='저장할 파일 경로를 적으세요',
    tokenizer=tokenizer,
    hub_model_id="허브에 올라갈 주소를 적으세요 예)madatnlp/rob-large-krmath2",
)
callbacks = [es, hub_callback]

/content/roberta-large-krmath2-classification is already a clone of https://huggingface.co/madatnlp/rob-large-krmath2. Make sure you pull the latest changes with `repo.git_pull()`.


In [21]:
train_step = len(train_idx) // BATCH_SIZE
val_step = len(valid_idx) // BATCH_SIZE

In [22]:
print(train_idx.shape)
print(valid_idx.shape)

(2538,)
(282,)


# 모델 학습  
데이터셋을 활용해서 학습할 때는 train_step과 validation_step을 설정해줘야 함을 유의하세요

In [23]:
history = model.fit(
    tf_train_ds,
    epochs=50,
    batch_size=BATCH_SIZE,
    callbacks=callbacks,
    validation_data=tf_val_ds,
    steps_per_epoch=train_step,
    validation_steps=val_step,
)

Epoch 1/50
6/9 [===================>..........] - ETA: 1s - loss: 1.8180WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0063s vs `on_train_batch_end` time: 15.1381s). Check your callbacks.


9/9 [==============================] - ETA: 0s - loss: 1.7479

Several commits (10) will be pushed upstream.


9/9 [==============================] - 233s 12s/step - loss: 1.7479 - val_loss: 1.5215
Epoch 2/50
9/9 [==============================] - 4s 450ms/step - loss: 1.1286 - val_loss: 0.7513
Epoch 3/50
9/9 [==============================] - 4s 447ms/step - loss: 1.2498 - val_loss: 0.9242
Epoch 4/50
9/9 [==============================] - 4s 450ms/step - loss: 1.0213 - val_loss: 0.7140
Epoch 5/50
9/9 [==============================] - 4s 448ms/step - loss: 0.8002 - val_loss: 0.6080
Epoch 6/50
9/9 [==============================] - 4s 450ms/step - loss: 0.5895 - val_loss: 0.3721
Epoch 7/50
9/9 [==============================] - 4s 449ms/step - loss: 0.4699 - val_loss: 0.3392
Epoch 8/50
9/9 [==============================] - 4s 450ms/step - loss: 0.3064 - val_loss: 0.2599
Epoch 9/50
9/9 [==============================] - 4s 448ms/step - loss: 0.2803 - val_loss: 0.2360
Epoch 10/50
9/9 [==============================] - 4s 451ms/step - loss: 0.2162 - val_loss: 0.3075
Epoch 11/50
9/9 [=============

Several commits (11) will be pushed upstream.
The progress bars may be unreliable.


Upload file tf_model.h5:   0%|          | 3.33k/1.25G [00:00<?, ?B/s]

remote: Enforcing permissions...        
remote: Allowed refs: all        
To https://huggingface.co/madatnlp/rob-large-krmath2
   7caa836..4f4a87e  main -> main

remote: Allowed refs: all        
To https://huggingface.co/madatnlp/rob-large-krmath2
   7caa836..4f4a87e  main -> main



In [24]:
tokenized = test.problem.apply(lambda x:tokenizer.encode(x))

In [25]:
len_token = tokenized.apply(len)

In [26]:
len_token.describe()

count    282.000000
mean      38.971631
std       13.810844
min       14.000000
25%       29.000000
50%       37.000000
75%       45.750000
max       95.000000
Name: problem, dtype: float64

# Test data에 Class label

In [27]:
problem = test.problem.to_list()
tokenized_problem = tokenizer(problem,max_length=95, padding='max_length', truncation=True)

In [28]:
tokenized_problem.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [29]:
testset = dict(
    input_ids=tokenized_problem['input_ids'],
    attention_mask=tokenized_problem['attention_mask']
)

In [30]:
AUTO = tf.data.AUTOTUNE
BATCH_SIZE=8

def prefetch_ds(ds):
    return ds.repeat().shuffle(2500).batch(BATCH_SIZE,num_parallel_calls=AUTO).prefetch(AUTO)

In [31]:
tf_testset = tf.data.Dataset.from_tensor_slices(tokenized_problem['input_ids'])
prefetched = prefetch_ds(tf_testset)

In [32]:
# 학습을 종료하고 모델 허브에 올렸던 모델 불러오기
model = TFAutoModelForSequenceClassification.from_pretrained("madatnlp/rob-large-krmath2")

Downloading:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at madatnlp/rob-large-krmath2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [33]:
pred = model.predict(test_set)

In [41]:
PRED = tf.argmax(pred['logits'],axis=-1)

In [43]:
inversed_label = label_encoder.inverse_transform(PRED)

In [44]:
inversed_label

array([7, 4, 1, 4, 1, 8, 8, 1, 1, 2, 1, 1, 1, 1, 8, 1, 1, 8, 8, 1, 8, 1,
       1, 8, 8, 3, 1, 8, 1, 6, 1, 4, 1, 6, 8, 8, 1, 1, 1, 7, 8, 4, 1, 1,
       1, 8, 2, 7, 5, 3, 1, 8, 7, 8, 6, 8, 1, 3, 8, 1, 8, 1, 8, 8, 8, 7,
       1, 8, 4, 6, 4, 3, 8, 8, 1, 1, 4, 8, 1, 6, 1, 7, 1, 1, 8, 1, 5, 3,
       1, 1, 1, 1, 7, 8, 1, 1, 1, 4, 5, 4, 8, 1, 6, 3, 1, 1, 1, 1, 3, 8,
       1, 1, 3, 1, 1, 4, 1, 1, 1, 8, 5, 8, 3, 8, 8, 8, 3, 1, 8, 1, 8, 8,
       4, 1, 3, 4, 1, 4, 2, 8, 1, 8, 7, 1, 6, 1, 4, 1, 8, 1, 8, 4, 1, 7,
       8, 8, 8, 4, 1, 6, 1, 8, 8, 1, 6, 1, 8, 8, 7, 8, 1, 8, 1, 3, 4, 1,
       8, 3, 1, 1, 5, 8, 8, 8, 1, 6, 8, 8, 1, 8, 4, 8, 6, 1, 1, 8, 3, 1,
       6, 8, 8, 7, 8, 1, 1, 5, 8, 8, 5, 4, 3, 1, 6, 8, 3, 8, 1, 1, 5, 8,
       1, 1, 8, 3, 7, 1, 8, 1, 5, 1, 1, 1, 1, 6, 1, 5, 5, 8, 1, 4, 8, 6,
       3, 3, 3, 1, 5, 7, 8, 1, 8, 7, 1, 7, 1, 8, 8, 1, 1, 7, 4, 1, 1, 6,
       7, 5, 8, 4, 8, 7, 4, 1, 1, 7, 3, 8, 8, 8, 8, 1, 2, 1])

In [45]:
test['class'] = inversed_label

In [ ]:
test.to_csv('파일을 저장할 경로/test_class.csv',index=False)